### Preprocess

In [ ]:
# /root/autodl-tmp/CIFAR100/Cifar_100_train_10fold.csv
# /root/autodl-tmp/SVHN/SVHN_train_10fold.csv
# /root/autodl-tmp/CUB_200_2011/CUB_200_2011_train_10fold.csv
# /root/autodl-tmp/Stanford_Cars/Standford_Cars_train_10fold.csv
# /root/autodl-tmp/Places_LT/Places_LT_train.csv
# /root/autodl-tmp/ip102_v1.1/IP102_train.csv
# /root/autodl-tmp/OfficeHome/OfficeHome_train_10fold.csv
# /root/autodl-tmp/PACS/PACS_train_10fold.csv

* CUB_200_2011

In [ ]:
import os
with open('/root/autodl-tmp/CUB_200_2011/image_class_labels.txt', 'r') as f:
    data1 = [line.strip().split()[-1] for line in f.readlines()]
with open('/root/autodl-tmp/CUB_200_2011/images.txt', 'r') as f:
    data2 = [os.path.join('/root/autodl-tmp/CUB_200_2011/images', line.strip().split()[-1]) for line in f.readlines()]
with open('/root/autodl-tmp/CUB_200_2011/train_test_split.txt', 'r') as f:
    data3 = [line.strip().split()[-1] for line in f.readlines()]

In [ ]:
import pandas as pd
import numpy as np
df_train = []
df_test = []
data1 = [int(e) for e in data1]
label_unique = list(np.unique(data1))
for l1, l2, l3 in zip(data1, data2, data3):
    if l3 == '1':
        df_train.append([l2, label_unique.index(l1)])
    else:
        df_test.append([l2, label_unique.index(l1)])
df_train = pd.DataFrame(df_train, columns=['image_path', 'label'])
df_test = pd.DataFrame(df_test, columns=['image_path', 'label'])
df_train.to_csv('/root/autodl-tmp/CUB_200_2011/CUB_200_2011_train.csv', index=False)
df_test.to_csv('/root/autodl-tmp/CUB_200_2011/CUB_200_2011_test.csv', index=False)
display(df_train)
df_test

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
df = pd.read_csv('/root/autodl-tmp/CUB_200_2011/CUB_200_2011_train.csv')
split = list(StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(df, df['label']))
for fold, (train_index, val_index) in enumerate(tqdm(split)):
    df.loc[val_index, 'fold'] = fold
df.to_csv('/root/autodl-tmp/CUB_200_2011/CUB_200_2011_train_10fold.csv', index=False)
df

* Stanford Cars

In [ ]:
import scipy.io as scio
from glob import glob
import numpy as np
import pandas as pd
import os
data = scio.loadmat('/root/autodl-tmp/Stanford_Cars/cars_annos.mat')['annotations']
df_train = []
df_test = []
for i in range(data.shape[1]):
    if data[0, i][-1][0, 0] == 1:
        df_train.append([os.path.join('/root/autodl-tmp/Stanford_Cars', data[0, i][0][0]), data[0, i][-2][0, 0]-1])
    else:
        df_test.append([os.path.join('/root/autodl-tmp/Stanford_Cars', data[0, i][0][0]), data[0, i][-2][0, 0]-1])
df_train = pd.DataFrame(df_train, columns=['image_path', 'label'])
df_test = pd.DataFrame(df_test, columns=['image_path', 'label'])
df_train.to_csv('/root/autodl-tmp/Stanford_Cars/Standford_Cars_train.csv', index=False)
df_test.to_csv('/root/autodl-tmp/Stanford_Cars/Standford_Cars_test.csv', index=False)
display(df_train)
df_test

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
df = pd.read_csv('/root/autodl-tmp/Stanford_Cars/Standford_Cars_train.csv')
split = list(StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(df, df['label']))
for fold, (train_index, val_index) in enumerate(tqdm(split)):
    df.loc[val_index, 'fold'] = fold
df.to_csv('/root/autodl-tmp/Stanford_Cars/Standford_Cars_train_10fold.csv', index=False)
df

* Oxford Flowers

In [ ]:
import scipy.io as scio
from glob import glob
import numpy as np
import pandas as pd
data1 = scio.loadmat('/root/autodl-tmp/Oxford_Flowers/imagelabels.mat')
data2 = scio.loadmat('/root/autodl-tmp/Oxford_Flowers/setid.mat')
train_index, valid_index, test_index = data2['trnid'], data2['valid'], data2['tstid']
labels = data1['labels'].astype('float')
images = glob('/root/autodl-tmp/Oxford_Flowers/jpg/*')
images = sorted(images)

In [ ]:
df_train = []
df_val = []
df_test = []
for i in train_index[0, :]-1:
    df_train.append([images[i], labels[0, i]-1])
df_train = pd.DataFrame(df_train, columns=['image_path', 'label'])
df_train['fold'] = 0
for i in valid_index[0, :]-1:
    df_val.append([images[i], labels[0, i]-1])
df_val = pd.DataFrame(df_val, columns=['image_path', 'label'])
df_val['fold'] = 1
for i in test_index[0, :]-1:
    df_test.append([images[i], labels[0, i]-1])
df_test = pd.DataFrame(df_test, columns=['image_path', 'label'])
df_train_val = pd.concat([df_train, df_val], axis=0).reset_index(drop=True)
df_train_val.to_csv('/root/autodl-tmp/Oxford_Flowers/Oxford_Flowers_train.csv', index=False)
df_test.to_csv('/root/autodl-tmp/Oxford_Flowers/Oxford_Flowers_test.csv', index=False)
display(df_train_val)
df_test

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
df = pd.read_csv('FGVC/Oxford_Flowers/Oxford_Flowers_train.csv')
df_1_shot = df.copy()
df_5_shot = df.copy()
df_10_shot = df.copy()
split1 = list(StratifiedKFold(n_splits=20, shuffle=True, random_state=1).split(df, df['label']))
split2 = list(StratifiedKFold(n_splits=4, shuffle=True, random_state=1).split(df, df['label']))
split3 = list(StratifiedKFold(n_splits=2, shuffle=True, random_state=1).split(df, df['label']))
for fold, (train_index, val_index) in enumerate(tqdm(split1)):
    df_1_shot.loc[val_index, 'fold'] = fold
for fold, (train_index, val_index) in enumerate(tqdm(split2)):
    df_5_shot.loc[val_index, 'fold'] = fold
for fold, (train_index, val_index) in enumerate(tqdm(split3)):
    df_10_shot.loc[val_index, 'fold'] = fold
df_1_shot.to_csv('FGVC/Oxford_Flowers/Oxford_Flowers_train_1_shot.csv', index=False)
df_5_shot.to_csv('FGVC/Oxford_Flowers/Oxford_Flowers_train_5_shot.csv', index=False)
df_10_shot.to_csv('FGVC/Oxford_Flowers/Oxford_Flowers_train_10_shot.csv', index=False)
df_10_shot.head()

* Cifar100

In [ ]:
from glob import glob
import pandas as pd
import numpy as np
import os
train_imgs = glob('/root/autodl-tmp/CIFAR100/TRAIN/*/*')
test_imgs = glob('/root/autodl-tmp/CIFAR100/TEST/*/*')
label2idx = os.listdir('/root/autodl-tmp/CIFAR100/TEST')
train_labels = list(map(lambda x:label2idx.index(x.split('/')[5]), train_imgs))
test_labels = list(map(lambda x:label2idx.index(x.split('/')[5]), test_imgs))
df_train = pd.DataFrame({'image_path': train_imgs, 'label': train_labels})
df_test = pd.DataFrame({'image_path': test_imgs, 'label': test_labels})
df_train.to_csv('/root/autodl-tmp/CIFAR100/Cifar_100_train.csv', index=False)
df_test.to_csv('/root/autodl-tmp/CIFAR100/Cifar_100_test.csv', index=False)
display(df_train)
df_test

In [ ]:
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
df = pd.read_csv('/root/autodl-tmp/CIFAR100/Cifar_100_train.csv')
split = list(StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(df, df['label']))
for fold, (train_index, val_index) in enumerate(tqdm(split)):
    df.loc[val_index, 'fold'] = fold
df.to_csv('/root/autodl-tmp/CIFAR100/Cifar_100_train_10fold.csv', index=False)
df

* SVHN

In [ ]:
import numpy as np
import os
import cv2
import pandas as pd
from tqdm.notebook import tqdm, trange

os.makedirs('/root/autodl-tmp/SVHN/train_images', exist_ok=True)
os.makedirs('/root/autodl-tmp/SVHN/test_images', exist_ok=True)
train_imgs = np.load('/root/autodl-tmp/SVHN/X_train.npy')
train_labels = np.load('/root/autodl-tmp/SVHN/y_train.npy')
test_imgs = np.load('/root/autodl-tmp/SVHN/X_test.npy')
test_labels = np.load('/root/autodl-tmp/SVHN/y_test.npy')
df_train = []
df_test = []
for i in trange(train_imgs.shape[-1]):
    save_path = f'/root/autodl-tmp/SVHN/train_images/{i}.jpg'
    cv2.imwrite(save_path, train_imgs[:, :, ::-1, i]*255)
    df_train.append([save_path, train_labels[i, 0]])
for i in trange(test_imgs.shape[-1]):
    save_path = f'/root/autodl-tmp/SVHN/test_images/{i}.jpg'
    cv2.imwrite(save_path, test_imgs[:, :, ::-1, i]*255)
    df_test.append([save_path, test_labels[i, 0]])
df_train = pd.DataFrame(df_train, columns=['image_path', 'label'])
df_test = pd.DataFrame(df_test, columns=['image_path', 'label'])
df_train.to_csv('/root/autodl-tmp/SVHN/SVHN_train.csv', index=False)
df_test.to_csv('/root/autodl-tmp/SVHN/SVHN_test.csv', index=False)
display(df_train)
df_test

In [ ]:
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
df = pd.read_csv('/root/autodl-tmp/SVHN/SVHN_train.csv')
split = list(StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(df, df['label']))
for fold, (train_index, val_index) in enumerate(tqdm(split)):
    df.loc[val_index, 'fold'] = fold
df.to_csv('/root/autodl-tmp/SVHN/SVHN_train_10fold.csv', index=False)
df

* PACS

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm, trange
import os

all_images = glob('/root/autodl-tmp/PACS/*/*/*')
train_imgs = [i for i in all_images if 'art_painting' not in i]
test_imgs = [i for i in all_images if 'art_painting' in i]
label2idx = os.listdir('/root/autodl-tmp/PACS/art_painting')
train_labels = list(map(lambda x:label2idx.index(x.split('/')[5]), train_imgs))
test_labels = list(map(lambda x:label2idx.index(x.split('/')[5]), test_imgs))
df_train = pd.DataFrame({'image_path': train_imgs, 'label': train_labels})
df_test = pd.DataFrame({'image_path': test_imgs, 'label': test_labels})
df_train.to_csv('/root/autodl-tmp/PACS/PACS_train.csv', index=False)
df_test.to_csv('/root/autodl-tmp/PACS/PACS_test.csv', index=False)
display(df_train)
df_test

In [ ]:
from sklearn.model_selection import StratifiedKFold

df = pd.read_csv('/root/autodl-tmp/PACS/PACS_train.csv')
split = list(StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(df, df['label']))
for fold, (train_index, val_index) in enumerate(tqdm(split)):
    df.loc[val_index, 'fold'] = fold
df.to_csv('/root/autodl-tmp/PACS/PACS_train_10fold.csv', index=False)
df

* OfficeHome

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm, trange
import os

all_images = glob('/root/autodl-tmp/OfficeHome/*/*/*')
train_imgs = [i for i in all_images if 'Real World' not in i]
test_imgs = [i for i in all_images if 'Real World' in i]
label2idx = os.listdir('/root/autodl-tmp/OfficeHome/Real World')
train_labels = list(map(lambda x:label2idx.index(x.split('/')[5]), train_imgs))
test_labels = list(map(lambda x:label2idx.index(x.split('/')[5]), test_imgs))
df_train = pd.DataFrame({'image_path': train_imgs, 'label': train_labels})
df_test = pd.DataFrame({'image_path': test_imgs, 'label': test_labels})
df_train.to_csv('/root/autodl-tmp/OfficeHome/OfficeHome_train.csv', index=False)
df_test.to_csv('/root/autodl-tmp/OfficeHome/OfficeHome_test.csv', index=False)
display(df_train)
df_test

In [ ]:
from sklearn.model_selection import StratifiedKFold

df = pd.read_csv('/root/autodl-tmp/OfficeHome/OfficeHome_train.csv')
split = list(StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(df, df['label']))
for fold, (train_index, val_index) in enumerate(tqdm(split)):
    df.loc[val_index, 'fold'] = fold
df.to_csv('/root/autodl-tmp/OfficeHome/OfficeHome_train_10fold.csv', index=False)
df

* Places_LT

In [ ]:
import os
import pandas as pd
from glob import glob
from tqdm.notebook import trange, tqdm
import numpy as np
str2idx = [i for i in range(365)]
with open('/root/autodl-tmp/Places_LT/Places_LT_train.txt', 'r') as f:
    data = f.readlines()
    train_imgs = []
    train_labels = []
    for i in range(len(data)):
        d = data[i].strip().split()[0]
        l = data[i].strip().split()[-1]
        d_splits = d.split('/')
        if len(d_splits) == 5:
            path = os.path.join('/root/autodl-tmp/Places_LT/train', d.split('/', 2)[-1].replace('/', '-', 1))
        else:
            path = os.path.join('/root/autodl-tmp/Places_LT/train', d.split('/', 2)[-1])
        train_imgs.append(path)
        train_labels.append(l)
        
with open('/root/autodl-tmp/Places_LT/Places_LT_val.txt', 'r') as f:
    data = f.readlines()
    val_imgs = []
    val_labels = []
    for i in range(len(data)):
        d = data[i].strip().split()[0]
        l = data[i].strip().split()[-1]
        d_splits = d.split('/')
        if len(d_splits) == 5:
            path = os.path.join('/root/autodl-tmp/Places_LT/train', d.split('/', 2)[-1].replace('/', '-', 1))
        else:
            path = os.path.join('/root/autodl-tmp/Places_LT/train', d.split('/', 2)[-1])
        val_imgs.append(path)
        val_labels.append(l)
        str2idx[int(l)] = path.split('/')[5]

test_imgs = glob('/root/autodl-tmp/Places_LT/val/*/*')
test_labels = list(map(lambda x:str2idx.index(x.split('/')[5]), test_imgs))

In [ ]:
df_train = pd.DataFrame({'image_path': train_imgs, 'label': train_labels})
df_val = pd.DataFrame({'image_path': val_imgs, 'label': val_labels})
df_test = pd.DataFrame({'image_path': test_imgs, 'label': test_labels})
df_train['fold'] = 0
df_val['fold'] = 1
df_train_val = pd.concat([df_train, df_val], axis=0)
df_train_val.to_csv('/root/autodl-tmp/Places_LT/Places_LT_train.csv', index=False)
df_test.to_csv('/root/autodl-tmp/Places_LT/Places_LT_test.csv', index=False)
display(df_train_val)
df_test

* IP102

In [ ]:
import pandas as pd
import os
with open('/root/autodl-tmp/ip102_v1.1/train.txt', 'r') as f:
    data = f.readlines()
    train_imgs = [os.path.join('/root/autodl-tmp/ip102_v1.1/images', i.strip().split()[0]) for i in data]
    train_labels = [i.strip().split()[-1] for i in data]
with open('/root/autodl-tmp/ip102_v1.1/val.txt', 'r') as f:
    data = f.readlines()
    val_imgs = [os.path.join('/root/autodl-tmp/ip102_v1.1/images', i.strip().split()[0]) for i in data]
    val_labels = [i.strip().split()[-1] for i in data]
with open('/root/autodl-tmp/ip102_v1.1/test.txt', 'r') as f:
    data = f.readlines()
    test_imgs = [os.path.join('/root/autodl-tmp/ip102_v1.1/images', i.strip().split()[0]) for i in data]
    test_labels = [i.strip().split()[-1] for i in data]

In [ ]:
import pandas as pd
import numpy as np
df_train = pd.DataFrame({'image_path': train_imgs, 'label': train_labels})
df_val = pd.DataFrame({'image_path': val_imgs, 'label': val_labels})
df_test = pd.DataFrame({'image_path': test_imgs, 'label': test_labels})
df_train['fold'] = 0
df_val['fold'] = 1
df_train_val = pd.concat([df_train, df_val], axis=0)
df_train_val.to_csv('/root/autodl-tmp/ip102_v1.1/IP102_train.csv', index=False)
df_test.to_csv('/root/autodl-tmp/ip102_v1.1/IP102_test.csv', index=False)
display(df_train_val)
df_test

* add

In [ ]:
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
df = pd.read_csv('/root/autodl-tmp/CIFAR100/Cifar_100_train.csv')
split = list(StratifiedKFold(n_splits=2, shuffle=True, random_state=1).split(df, df['label']))
for fold, (train_index, val_index) in enumerate(tqdm(split)):
    df.loc[val_index, 'fold'] = fold
df.to_csv('/root/autodl-tmp/CIFAR100/Cifar_100_train_2fold.csv', index=False)
df